In [1]:
%matplotlib inline

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer

from sklearn.linear_model import LogisticRegression

# Model Training and Improvement
## Live Demos

In [3]:
diabetes_data = pd.read_csv("data/diabetic_data.csv")

In [4]:
diabetes_data

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [5]:
diabetes_data.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

Виждаме кои са id-та

In [6]:
diabetes_data.columns[diabetes_data.columns.str.contains("_id")]

Index(['encounter_id', 'admission_type_id', 'discharge_disposition_id',
       'admission_source_id'],
      dtype='object')

Можем да видим за някое лекарство как му е променяна дозата :

In [7]:
diabetes_data.pioglitazone.unique()

array(['No', 'Steady', 'Up', 'Down'], dtype=object)

Ако имаме такъв dataset и ако се опитаме да направим модел върху него, ще гръмне, щото не съдържа само числа. Тъй като в случая искаме да предскажем readmitted, представяме си, че единствения алгоритъм, който знаем за класификация, е LogisticRegression.

In [8]:
logistic_regression = LogisticRegression()

Правим fit, ама няма да работи заради текстовите данни. 

In [9]:
logistic_regression.fit(diabetes_data.drop(columns = "readmitted"), diabetes_data.readmitted)

ValueError: could not convert string to float: 'Caucasian'

Затова правим обработка на данните. Някои от  Id-тата например са категорийни променливи. 

Използваме pd.get_dummies за да превърнем матрицата в числа.

In [11]:
pd.get_dummies(diabetes_data)

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted_<30,readmitted_>30,readmitted_NO
0,2278392,8222157,6,25,1,1,41,0,1,0,...,False,True,False,False,True,True,False,False,False,True
1,149190,55629189,1,1,7,3,59,0,18,0,...,False,True,False,True,False,False,True,False,True,False
2,64410,86047875,1,1,7,2,11,5,13,2,...,False,True,False,False,True,False,True,False,False,True
3,500364,82442376,1,1,7,2,44,1,16,0,...,False,True,False,True,False,False,True,False,False,True
4,16680,42519267,1,1,7,1,51,0,8,0,...,False,True,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,1,3,7,3,51,0,16,0,...,False,True,False,True,False,False,True,False,True,False
101762,443847782,74694222,1,4,5,5,33,3,18,0,...,False,True,False,False,True,False,True,False,False,True
101763,443854148,41088789,1,1,7,1,53,0,9,1,...,False,True,False,True,False,False,True,False,False,True
101764,443857166,31693671,2,3,7,10,45,2,21,0,...,False,True,False,True,False,False,True,False,False,True


Имаме толкова много колони, защото всяка от категорийните променливи стават по 4 колони. 

In [14]:
diabetes_attributes = diabetes_data.drop(columns="readmitted")
diabetes_target = diabetes_data["readmitted"]

In [15]:
diabetes_attributes_dummies = pd.get_dummies(diabetes_attributes)

In [16]:
logistic_regression = LogisticRegression()

In [17]:
logistic_regression.fit(diabetes_attributes_dummies, diabetes_target)

LogisticRegression()

In [18]:
logistic_regression.coef_

array([[-3.31701395e-09, -3.86905107e-09, -1.15814534e-15, ...,
        -3.26369841e-16, -1.63318872e-16, -3.02903976e-16],
       [-1.50329991e-10,  4.51003945e-09, -1.47130836e-16, ...,
        -1.14848644e-16, -1.11876233e-16,  7.41720589e-17],
       [ 3.46734394e-09, -6.40988385e-10,  1.30527618e-15, ...,
         4.41218484e-16,  2.75195106e-16,  2.28731917e-16]])

Може да ползваме pipelines предварително да обработим данните (скалиране, polynomia features). За щастие Sklearn също има. Можем да закачим цялата логика на данни към модела и да я експортнем заедно с него. Това са sklearn pipelines. Това ни гарантира ако трябва да променим нещо по данните да имаме репродуцируем модел с данни, да му подадем суровите, да си пуснем pipeline-a, той да си ги обработи и да ги подаде на модела.

Взимаме скалирането... 

In [19]:
scaler = MinMaxScaler()

In [20]:
diabetes_attributes_scaled = scaler.fit_transform(diabetes_attributes_dummies)

In [21]:
logistic_regression.fit(diabetes_attributes_scaled, diabetes_target)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
logistic_regression.score(diabetes_attributes_scaled, diabetes_target)

0.5952086158441916

Виждаме какъв е бил оргиналния 

In [23]:
logistic_regression.score(diabetes_attributes_dummies, diabetes_target)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.5387162706601419

Разликата не е голяма.
Започваме да работим с pipelines. Организираме импортите.

In [27]:
pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("model", LogisticRegression())
])

In [29]:
pipeline #Когато поискаме Pipeline и jupyter ни я върне, получаваме визуализация

Pipeline(steps=[('scaler', MinMaxScaler()), ('model', LogisticRegression())])

После правим pipeline.fit върху ОРИГИНАЛНИТЕ ДАННИ.
За по-бързо взимаме sample от данните. Използваме random_state за да получим всеки път един и същи редове в резултата

In [30]:
sample_data = diabetes_data.sample(5000, random_state=42)

In [31]:
sample_data

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
35956,110939484,19274094,Caucasian,Female,[70-80),?,1,1,6,11,...,No,Steady,No,No,No,No,No,No,Yes,NO
60927,170328306,65634327,Caucasian,Male,[50-60),?,1,1,1,1,...,No,No,No,No,No,No,No,No,Yes,NO
79920,245688426,100657359,Caucasian,Female,[60-70),?,3,6,1,4,...,No,No,No,No,No,No,No,No,Yes,NO
50078,150826224,83144448,Caucasian,Male,[30-40),?,2,1,1,12,...,No,No,No,No,No,No,No,No,Yes,>30
44080,135993852,65234214,AfricanAmerican,Female,[60-70),?,1,2,7,1,...,No,No,No,No,No,No,No,No,Yes,<30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35562,110096790,33502212,Caucasian,Male,[50-60),?,6,1,7,12,...,No,Down,No,No,No,No,No,Ch,Yes,NO
98563,402583472,141357506,AfricanAmerican,Male,[50-60),?,7,3,7,6,...,No,Steady,No,No,No,No,No,No,Yes,NO
88066,282442506,45759951,Caucasian,Female,[40-50),?,1,1,7,3,...,No,No,No,No,No,No,No,No,Yes,NO
55955,161139018,112510251,Caucasian,Female,[40-50),?,5,1,1,3,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [35]:
#Виждаме, че има само булеви променливи и числа. 
pd.get_dummies(sample_data).dtypes

encounter_id                int64
patient_nbr                 int64
admission_type_id           int64
discharge_disposition_id    int64
admission_source_id         int64
                            ...  
diabetesMed_No               bool
diabetesMed_Yes              bool
readmitted_<30               bool
readmitted_>30               bool
readmitted_NO                bool
Length: 1255, dtype: object

In [36]:
pd.get_dummies(sample_data).astype(int)

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-pioglitazone_No,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted_<30,readmitted_>30,readmitted_NO
35956,110939484,19274094,1,1,6,11,68,0,20,0,...,1,1,1,0,1,0,1,0,0,1
60927,170328306,65634327,1,1,1,1,20,0,7,0,...,1,1,1,0,1,0,1,0,0,1
79920,245688426,100657359,3,6,1,4,21,3,23,1,...,1,1,1,0,1,0,1,0,0,1
50078,150826224,83144448,2,1,1,12,28,0,19,0,...,1,1,1,0,1,0,1,0,1,0
44080,135993852,65234214,1,2,7,1,21,0,6,0,...,1,1,1,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35562,110096790,33502212,6,1,7,12,65,2,19,0,...,1,1,1,1,0,0,1,0,0,1
98563,402583472,141357506,7,3,7,6,61,1,14,0,...,1,1,1,0,1,0,1,0,0,1
88066,282442506,45759951,1,1,7,3,28,6,6,0,...,1,1,1,0,1,0,1,0,0,1
55955,161139018,112510251,5,1,1,3,27,0,10,1,...,1,1,1,1,0,0,1,0,0,1


Това става даже само без ние да го правим.

In [37]:
sample_attributes = sample_data.drop(columns="readmitted")
sample_target = sample_data["readmitted"]

In [38]:
sample_attributes = pd.get_dummies(sample_attributes)

In [39]:
pipeline.fit(sample_attributes, sample_target)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('scaler', MinMaxScaler()), ('model', LogisticRegression())])

Добавяме OneHotEncoder

In [42]:
ohe = OneHotEncoder()

In [43]:
ohe.fit(sample_attributes)

OneHotEncoder()

Виждаме categories, feature names

In [44]:
ohe.feature_names_in_

array(['encounter_id', 'patient_nbr', 'admission_type_id', ...,
       'change_No', 'diabetesMed_No', 'diabetesMed_Yes'], dtype=object)

In [45]:
ohe.categories_

[array([   325848,   1139226,   1212006, ..., 443730002, 443775086,
        443824292], dtype=int64),
 array([    10827,     15849,     27315, ..., 186774602, 187042703,
        189502619], dtype=int64),
 array([1, 2, 3, 5, 6, 7, 8], dtype=int64),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 14, 15, 17, 18, 22, 23,
        24, 25, 28], dtype=int64),
 array([ 1,  2,  3,  4,  5,  6,  7,  9, 17, 20], dtype=int64),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       dtype=int64),
 array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  

OneHotEncoder е клас от библиотеката scikit-learn в Python, който се използва за преобразуване на категориални данни в числови, чрез техниката one-hot encoding. По този начин всяка уникална стойност на категориалната променлива се превръща в нова бинарна променлива (колонка), съдържаща 0 или 1.

Как работи?
При one-hot encoding, ако имаш колонка с N уникални категории, то за всяка категория се създава нова колонка, която ще съдържа:

1, ако редът съответства на дадената категория,
0, ако не съответства.

 Може да използваме sklearn column selector. Това и column transformer - операции, чрез които  можем да вземем колони които ни интересуват, и да правим собствени трансформации върху тях. Така например взимаме някакъв dataset, от който си избираме част от колоните и ги прекарваме през pipeline, и този pipeline е в CountTransformer. Върху други колони имаме друг pipeline, който си е за тях. Върху трети - трети и т.н....това е начинът да комбинираме по-сложна работа с данни и sklearn. CountTransformer - избери тези колони и направи тези трансформации върху тях.

Ще направим CountTransformer върху колоните, които стойности са стрингове.

In [47]:
sample_data.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [48]:
sample_data.dtypes[sample_data.dtypes == np.object_].index

Index(['race', 'gender', 'age', 'weight', 'payer_code', 'medical_specialty',
       'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [49]:
columns = sample_data.dtypes[sample_data.dtypes == np.object_].index.values

In [50]:
columns

array(['race', 'gender', 'age', 'weight', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone',
       'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide',
       'citoglipton', 'insulin', 'glyburide-metformin',
       'glipizide-metformin', 'glimepiride-pioglitazone',
       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
       'diabetesMed', 'readmitted'], dtype=object)

Разкарваме "readmitted"

In [59]:
categorical_columns = columns[:-1]

има property "reminder" на CountTransformer, който указва какво правим с другите properties.

Взимаме numerical columns, като изключваме няколко...

In [60]:
numerical_columns = [
    'admission_type_id', 'discharge_disposition_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
    'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses'
]

In [63]:
preprocessor = ColumnTransformer([
    ("categorical", OneHotEncoder(), categorical_columns),
    ("numerical", MinMaxScaler(), numerical_columns)
], remainder = "passthrough")

In [64]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('categorical', OneHotEncoder(),
                                 array(['race', 'gender', 'age', 'weight', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazon...
       'glipizide-metformin', 'glimepiride-pioglitazone',
       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change'],
      dtype=object)),
                                ('numerical', MinMaxScaler(),
                                 ['admission_type_id',
                                  'discharge_disposition_id',
                                  'time_in_hospital', 'num_lab_procedures',
                                  'num_procedures', 'num_medications',
                                  'number_outpatient', 'number_emergency',
                                  'number_inpatient', 'number_diagnoses'])])

ще искаме числените колони да вземем логаритъм от тях и да ги скалираме. Имаме FunctionTransformer който приема функция.

In [67]:
log_transformer = FunctionTransformer(np.log10)

In [68]:
number_processor = Pipeline([
    ("log_transformer", FunctionTransformer(np.log)),
    ("minmax", MinMaxScaler())
])

In [69]:
preprocessor = ColumnTransformer([
    ("categorical", OneHotEncoder(), categorical_columns),
    ("numerical", number_processor, numerical_columns)
], remainder = "passthrough")

In [70]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('categorical', OneHotEncoder(),
                                 array(['race', 'gender', 'age', 'weight', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazon...
       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change'],
      dtype=object)),
                                ('numerical',
                                 Pipeline(steps=[('log_transformer',
                                                  FunctionTransformer(func=<ufunc 'log'>)),
                                                 ('minmax', MinMaxScaler())]),
                                 ['admission_type_id',
                                  'discharge_disposition_id',
                                  'time_in_hospital', 'num_lab_procedures',
                                  'num_procedures', 'num_medications',
                                  'number_outpatient', 'number_emergency',
                                  'number_inpatient', 'number_diagnoses'])])

Искаме и да имаме Estimator

In [72]:
pipeline = Pipeline(
    [("preprocess", preprocessor),
    ("classifier", LogisticRegression())
])

In [73]:
pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  array(['race', 'gender', 'age', 'weight', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburi...
                                                 ('numerical',
                                                  Pipeline(steps=[('log_transformer',
                                                                   FunctionTransformer(func=<ufunc 'log'>)),
                                                                  ('minmax',
                                                                   MinMaxScaler())]),
                                                  ['admission_type_id',
                                                   'discharge_disposition_id',
                                                   'time_in_hospital',
                                                   'num_lab_procedures',
                                                   'num_procedures',
                                                   'num_medications',
                                                   'number_outpatient',
                                                   'number_emergency',
                                                   'number_inpatient',
                                                   'number_diagnoses'])])),
                ('classifier', LogisticRegression())])

За целите на примера разкарваме remainder = "passthrough" на preprocessor

In [74]:
preprocessor = ColumnTransformer([
    ("categorical", OneHotEncoder(), categorical_columns),
    ("numerical", number_processor, numerical_columns)
])

In [75]:
pipeline = Pipeline(
    [("preprocess", preprocessor),
    ("classifier", LogisticRegression())
])

In [76]:
pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  array(['race', 'gender', 'age', 'weight', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'piogl...
                                                 ('numerical',
                                                  Pipeline(steps=[('log_transformer',
                                                                   FunctionTransformer(func=<ufunc 'log'>)),
                                                                  ('minmax',
                                                                   MinMaxScaler())]),
                                                  ['admission_type_id',
                                                   'discharge_disposition_id',
                                                   'time_in_hospital',
                                                   'num_lab_procedures',
                                                   'num_procedures',
                                                   'num_medications',
                                                   'number_outpatient',
                                                   'number_emergency',
                                                   'number_inpatient',
                                                   'number_diagnoses'])])),
                ('classifier', LogisticRegression())])

Това е Estimator - има си fit, predict, score. Може да го използваме като другите алгоритми в sklearn. Но имаме не само LogisticRegression, ами и всичко, което върви с него.

Всеки един pipeline има steps.

In [77]:
pipeline.steps[0] #Също pipeline, който има steps.

('preprocess',
 ColumnTransformer(transformers=[('categorical', OneHotEncoder(),
                                  array(['race', 'gender', 'age', 'weight', 'payer_code',
        'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
        'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
        'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
        'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone',
        'acar...
        'metformin-rosiglitazone', 'metformin-pioglitazone', 'change'],
       dtype=object)),
                                 ('numerical',
                                  Pipeline(steps=[('log_transformer',
                                                   FunctionTransformer(func=<ufunc 'log'>)),
                                                  ('minmax', MinMaxScaler())]),
                                  ['admission_type_id',
                                   'discharge_disposition_id',
                            

In [78]:
pipeline.steps[1]

('classifier', LogisticRegression())

Идеята е да се започне с прост pipeline и да го усложняваме итеративно и да получим много сложен pipeline. В момента, в който станат сложни нещата, ни трябва възпроизводимост - repeatability.
Друго предимство е - това е нещо, което приема съвсем сурови данни - така че като го пуснем, работи самостоятелно. 

как запазваме модели - pickle - библиотека за сериализация и десериализация в Python. 

При обучение на модел ще е добре да запазим резултатите на последния модел. 

In [79]:
pipeline.steps[0][1]

ColumnTransformer(transformers=[('categorical', OneHotEncoder(),
                                 array(['race', 'gender', 'age', 'weight', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone',
       'acar...
       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change'],
      dtype=object)),
                                ('numerical',
                                 Pipeline(steps=[('log_transformer',
                                                  FunctionTransformer(func=<ufunc 'log'>)),
                                                 ('minmax', MinMaxScaler())]),
                                 ['admission_type_id',
                                  'discharge_disposition_id',
                                  'time_in_hospital', 'num_lab_procedures',
                                  'num_procedures', 'num_medications',
                                  'number_outpatient', 'number_emergency',
                                  'number_inpatient', 'number_diagnoses'])])

като направим веднъж fit, после много пъти може да направим predict. 

Започваме на чисто.

In [80]:
diabetes_data = pd.read_csv("data/diabetic_data.csv")

In [82]:
sample_data = diabetes_data.sample(5000, random_state=12341234)
attributes = sample_data.drop(columns = ["readmitted"])
target = sample_data.readmitted

In [83]:
pipeline.fit(attributes, target)

C:\Users\HP\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


ValueError: Input X contains infinity or a value too large for dtype('float64').

Machine Learning е процеса, който следваме, за да получим правилни апроксиматори, които работят на практика. Работят на практика обаче има много измерения. Първото нещо, което трябва да знаем, е как ще измерваме колко добре работи нашия модел. Да предположим, че имаме метрика от 0 до 1. Не трябва да бъркаме loss функция от метрика. Каква е метриката - зависи от задачата, често метриката е комплексна. 
Имаме метрика, която работи. Ако имаме модел, който е постигнал 82% - въпросът е какво е това 82%? Може ли да се стигне 100%? Не съвсем.
Имаме случайни променливи...имаме процес, който не знаем какъв е - затова му присвояваме случайни променливи. Има вътрешен шум в данните, вътрешни вариабилности. - irreducible error, noise, baesian optimal error. Тоест в нашите данни ще има нещо, дето е непостижимо. Не знаем и не можем да измерим колко е максималният достижим резултат. Разликата между това което сме постигнали - тези 82 процента, и това, което може да се постигне, се дължи основно на 2 неща.. Едното е случайна грешка - случайности в данните - случайно е станало нещо, дето не сме очаквали. Това се нарича "variance", "statistical error". Другото се нарича "bias", "assumption error" - грешка от предположения - заради лош избор на данни, заради лош процес на обучение и т.н. Тези грешки могат да бъдат оценявани по различен начин. 
Variance е свързано със хаотично, непредвидимо представяне, не случайно.

Правим някаква метрика за това колко е близко алгоритъма до данните - например Mean Squared Error. Най-десния алгоритъм е перфектния. Имаме предположение, че данните могат да бъдат описани перфектно с полином от 6-та степен. Ама алгоритъма ако получи нови данни, ще сбърка и грешката ще е голяма. Но това са нови данни, които сме нямали преди. Друго нещо е, че това е много хаотично поведение. - при много  малки разлики във входа изхода става тотално различен. Макар че на хартия това е много добър алгоритъм, той не се справя с нови данни. Левият има high bias, десният има high variance. 

Затова въпросът на ML e какво работи достатъчно добре? Не е trade off - trade off означава намаляйки едното, да увеличаваме другото. За средната графика е кофти предположението, когато е голям размера на къщите - ама това означава, че нашият модел не познава данни за къщи с много голям размер - това не е ли bias? Тука може би сме направили екстраполация, ама не работи. 
Най-просто казано - high bias означава алгоритъм, който не познава дори данните, които сме му показали. Очакваме да се справи по-добре на тях и той още бърка
High variance - познава данните добре, но е доста хаотичен.
Какво да постигнем такъв tradeoff - правим - да вкараме наш bias. какво означава това? Да вкараме допълнително предположение...
Ако алгоритъмът ми е слаб, ще намерим начин да го направим по-силен, да му дадем повече свобода да варира. 
Връщането назад става с настройки. Те са различните видове. Специално за линейната и логистичната регресия имаме регуляризация - тя прави моделиращата ни функция по-регулярна - тоест да прави по-малко чупки. 
Например имаме loss функция - да речем Mean Squared Error и тя дава някакви резултати. В някакъв момент имаме 0, ама не ни върши работа. 
Полином от 6 степен има 7 параметъра. Решаваме че 7 параметъра са много и ги намаляме. Параметрите са нещо, което се променя в процеса на обучение. Така че може да е 0.x^5, 0.x^4, ама може и да ги намалим като числа. Много големи числа означава, че с малки промени в х y се променя много бързо. 
Така че взимаме тези тегловни коефициенти и ги добавяме към loss функцията. 
Тегловните коефициенти са много на брой числа. Какво означава да ги добавя към loss функцията - нищо не означава. Това, което може да използваме, е някаква мярка за големината на тези числа - при полином от 6-та степен са 7. Може да ги напишем в едно векторче и на тоя вектор да му вземем дължината . 2:14

